In [3]:
import os
from uvars import dataops_dir
import sys 
sys.path.append(dataops_dir)
from dataops import tileops
from glob import glob

In [4]:
vpath = "/home/ljp238/Downloads/BLOCKsV/SA.gpkg"
gridpath = "/home/ljp238/Downloads/FABDEM_v1-2_tiles.geojson"
gee_download_path = "/media/ljp238/12TBWolf/ARCHIEVE/GEEDownload/"
ps = 256
st = 256
cpus = 20

In [5]:
# utilenames, utilebboxs, xgdf = tileops.get_tilenames_from_global_grid_vector(vpath, gridpath)
# print("Tile names:", len(utilenames))

In [11]:
from tqdm import tqdm

In [12]:
def create_patches_from_vblock_pipeline(gee_download_path, vpath, gridpath, ps=256, st=256, cpus=20):
    utilenames, utilebboxs, xgdf = tileops.get_tilenames_from_global_grid_vector(vpath, gridpath)
    print("Tile names:", len(utilenames))

    gpkg_block_dir = os.path.join(gee_download_path, "SA", "GPKG")
    tif_block_dir = os.path.join(gee_download_path, "SA", "TIF")
    os.makedirs(gpkg_block_dir, exist_ok=True)
    os.makedirs(tif_block_dir, exist_ok=True)

    # Step 1: Create GPKGs
    outputs = tileops.processing_inpar(xgdf, utilenames, gpkg_block_dir, max_workers=cpus)
    gpkg_files = glob(f"{gpkg_block_dir}/*/*.gpkg")
    print("Total GPKG files created:", len(gpkg_files))

    # Step 2: Create Dummy TIFs
    dummy_tifs = tileops.create_dummy_rasters_inpar(gpkg_files, tif_block_dir, num_workers=cpus)

    # Step 3: Patchify each TIF with a progress bar
    for tifi in tqdm(dummy_tifs, desc="Processing TIFs with patchify_workflow"):
        tileops.patchify_workflow(tifi, ps, st, cpus=cpus)

In [13]:
create_patches_from_vblock_pipeline(gee_download_path=gee_download_path,
                                    vpath=vpath,gridpath=gridpath,
                                    ps=ps,st=st,cpus=cpus)

Tile names: 1855


Processing tiles (Parallel): 100%|██████████| 1855/1855 [00:00<00:00, 5961.26it/s]


Total GPKG files created: 1855


Processing TIFs with patchify_workflow: 100%|██████████| 1855/1855 [2:16:13<00:00,  4.41s/it]  
